# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
# Adding "keep_default_na=False" to avoid country code of "NA"(Nambia) getting converted to NaN in the dataset
city_data_df = pd.read_csv("../output_data/cities.csv", keep_default_na=False)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aginskoye,51.1000,114.5300,14.06,43,2,2.78,RU,1696996583
1,1,cedar city,37.6775,-113.0619,13.92,42,2,0.45,US,1696996930
2,2,redwood falls,44.5394,-95.1169,5.97,61,0,0.00,US,1696997008
3,3,hammerfest,70.6634,23.6821,-0.75,69,22,3.09,NO,1696996564
4,4,mahibadhoo,3.7833,72.9667,28.15,74,100,6.79,MV,1696997119


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
world_city_map = city_data_df.hvplot.points("Lng", "Lat",  color = "City",
                                             size = "Humidity", alpha = 0.5, tiles=True,
                                             frame_width = 700, frame_height = 500 )

# Display the map
world_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
city_filter = (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 20) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] < 5)
ideal_vacation_cities_df = city_data_df.loc[city_filter]

# Drop any rows with null values
ideal_vacation_cities_df.dropna()

# Display sample data
ideal_vacation_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,tanhacu,-14.0214,-41.2481,20.33,79,0,3.03,BR,1696997134
81,81,ath thawrah,35.8469,38.5443,20.66,68,0,3.74,SY,1696997135
123,123,stephenville,32.2207,-98.2023,21.80,67,0,4.12,US,1696997120
153,153,mancio lima,-7.6142,-72.8958,25.04,94,0,0.00,BR,1696997148
168,168,ierapetra,35.0100,25.7353,23.41,45,0,1.79,GR,1696997152
173,173,wajir,1.7471,40.0573,23.98,73,0,2.06,KE,1696997152
210,210,yazd,31.8972,54.3675,20.06,24,0,2.06,IR,1696997159
241,241,kourou,5.1552,-52.6478,24.03,83,0,3.11,GF,1696997166
247,247,buka,40.8108,69.1986,20.63,25,4,3.69,UZ,1696997167
259,259,al hufuf,25.3646,49.5653,26.19,25,0,1.54,SA,1696997169


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
76,tanhacu,BR,-14.0214,-41.2481,79,
81,ath thawrah,SY,35.8469,38.5443,68,
123,stephenville,US,32.2207,-98.2023,67,
153,mancio lima,BR,-7.6142,-72.8958,94,
168,ierapetra,GR,35.0100,25.7353,45,
173,wajir,KE,1.7471,40.0573,73,
210,yazd,IR,31.8972,54.3675,24,
241,kourou,GF,5.1552,-52.6478,83,
247,buka,UZ,40.8108,69.1986,25,
259,al hufuf,SA,25.3646,49.5653,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty & 
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params = params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tanhacu - nearest hotel: Tanhaçu Palace
ath thawrah - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
mancio lima - nearest hotel: No hotel found
ierapetra - nearest hotel: No hotel found
wajir - nearest hotel: Kilimanjaro
yazd - nearest hotel: هتل سنتی مهر
kourou - nearest hotel: SCI Horizon
buka - nearest hotel: No hotel found
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
mana - nearest hotel: Le Samana
smara - nearest hotel: Hôtel Amine فندق الأمين
kas - nearest hotel: KEKOVA OTEL
miguel ahumada - nearest hotel: Hotel Mexico
gimcheon - nearest hotel: 김천파트호텔 (Kimcheon Park Hotel)
jijel - nearest hotel: Hôtel Louiza
tan-tan - nearest hotel: Sable d´or
carutapera - nearest hotel: No hotel found
zuwarah - nearest hotel: فندق الدلفين
hurricane - nearest hotel: Wingate by Wyndham Hurricane Near Zion National Park
yeosu - nearest hotel: Am Ort Hotel
ma'rib - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
b

,City,Country,Lat,Lng,Humidity,Hotel Name
76,tanhacu,BR,-14.0214,-41.2481,79,Tanhaçu Palace
81,ath thawrah,SY,35.8469,38.5443,68,No hotel found
123,stephenville,US,32.2207,-98.2023,67,No hotel found
153,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
168,ierapetra,GR,35.0100,25.7353,45,No hotel found
173,wajir,KE,1.7471,40.0573,73,Kilimanjaro
210,yazd,IR,31.8972,54.3675,24,هتل سنتی مهر
241,kourou,GF,5.1552,-52.6478,83,SCI Horizon
247,buka,UZ,40.8108,69.1986,25,No hotel found
259,al hufuf,SA,25.3646,49.5653,25,Al Muhaidab Residence Al Ahsa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_details_map = hotel_df.hvplot.points("Lng", "Lat",  color = "City",
                                             size = "Humidity", alpha = 0.5, tiles=True,
                                             frame_width = 700, frame_height = 500,
                                             hover_cols = ["Hotel Name", "Country"]
                                          )

# Display the map
hotel_details_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)